In [66]:
import pandas as pd
import numpy as np
import datetime

In [67]:
# importando dados
df = pd.read_csv('data\gastos_presidenciais.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113340 entries, 0 to 113339
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ano                  113340 non-null  int64  
 1   presidente           113340 non-null  object 
 2   data_pagamento       113340 non-null  object 
 3   cpf_servidor         113340 non-null  object 
 4   cpf_cnpj_fornecedor  113340 non-null  object 
 5   nome_fornecedor      113340 non-null  object 
 6   tipo                 113340 non-null  object 
 7   subelemento_despesa  113340 non-null  object 
 8   cdic                 113340 non-null  object 
 9   valor                113339 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 8.6+ MB


In [68]:
# convertendo coluna para data
df['data_pagamento'] = pd.to_datetime(df['data_pagamento'])
df['mes'] = pd.DatetimeIndex(df['data_pagamento'])
df['mes'] = df['data_pagamento'].dt.month_name()

In [69]:
# criando coluna com mês abreviado e ano
df['mes'] = df['data_pagamento'].apply(lambda x: datetime.datetime.strftime(x, '%b'))
df['date'] = pd.to_datetime(df['mes'] + '/' + df['ano'].astype(str), format='%b/%Y')

In [70]:
# importando dados da inflação do IBGE e deletando colunas desnecessárias
infl = pd.read_excel('data\Tabela 1737.xlsx')
infl.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis = 1, inplace=True)
infl.columns

Index(['mes', 'ano', 'infl'], dtype='object')

In [71]:
# renomeando colunas e convertendo para data
infl.mes.replace(['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], inplace=True)
infl['date'] = pd.to_datetime(infl['mes'] + '/' + infl['ano'].astype(str), format='%b/%Y')

In [72]:
# separando apenas colunas necessárias e unindo com o df
infl2 = infl[['infl', 'date']]
df_merged = pd.merge(df, infl2, on = 'date')

In [73]:
# criando coluna com valor atualizado pela inflação
df_merged['valor_atualizado'] = (df_merged['infl'].iloc[-1] / df_merged['infl']) * df_merged['valor']
df_merged

,ano,presidente,data_pagamento,cpf_servidor,cpf_cnpj_fornecedor,nome_fornecedor,tipo,subelemento_despesa,cdic,valor,mes,date,infl,valor_atualizado
0,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1696.90,Jan,2003-01-01,2085.68,5267.290918
1,2003,Lula,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,1336.60,Jan,2003-01-01,2085.68,4148.895657
2,2003,Lula,2003-01-02,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,1396.43,Jan,2003-01-01,2085.68,4334.611972
3,2003,Lula,2003-01-03,***.004.131-**,00592717000170,FLORES ALVORADA,C,OUTROS SERVI€OS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,8585.00,Jan,2003-01-01,2085.68,26648.413299
4,2003,Lula,2003-01-03,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COM RCIO DE MATERIAIS PARA L...,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,251.50,Jan,2003-01-01,2085.68,780.672795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113335,2022,Bolsonaro,2022-12-03,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,179.50,Dec,2022-12-01,6474.09,179.500000
113336,2022,Bolsonaro,2022-12-03,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,54.95,Dec,2022-12-01,6474.09,54.950000
113337,2022,Bolsonaro,2022-12-04,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUI€ÇO,C,GENEROS DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,518.07,Dec,2022-12-01,6474.09,518.070000
113338,2022,Bolsonaro,2022-12-04,***.136.591-**,12550441000146,RESTAURANTE SOHO,C,FORNECIMENTO DE ALIMENTA€ÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,330.52,Dec,2022-12-01,6474.09,330.520000


In [74]:
# agrupando por presidente e ano e somando os valores
gastos_pr = df_merged.groupby(['presidente', 'ano'])['valor_atualizado'].sum().sort_values(ascending=False).reset_index()
gastos_pr['valor_atualizado'] = gastos_pr['valor_atualizado'].div(1000000).round(2)
gastos_pr.sort_values(by=['ano'], ascending = True).reset_index(drop=True)

,presidente,ano,valor_atualizado
0,Lula,2003,15.84
1,Lula,2004,18.28
2,Lula,2005,13.56
3,Lula,2006,12.48
4,Lula,2007,9.34
5,Lula,2008,13.85
6,Lula,2009,11.88
7,Lula,2010,13.59
8,Dilma,2011,8.92
9,Dilma,2012,8.50


In [75]:
# resumo dos gastos por presidente
gastos_pr_grp = gastos_pr.groupby(['presidente'])['valor_atualizado'].sum().reset_index().sort_values(by=['valor_atualizado'], ascending = False).reset_index(drop=True)
gastos_pr_grp

,presidente,valor_atualizado
0,Lula,108.82
1,Dilma,57.47
2,Bolsonaro,31.54
3,Temer,11.62


In [76]:
# número de anos que cada presidente esteve no poder
mandato = 4
mandato_pr = df_merged.groupby('presidente')['ano'].nunique().sort_values(ascending=True).reset_index()
mandato_pr['mandatos'] = mandato_pr['ano'] / mandato
mandato_pr

,presidente,ano,mandatos
0,Temer,2,0.5
1,Bolsonaro,4,1.0
2,Dilma,6,1.5
3,Lula,8,2.0


In [77]:
# valor médio gasto por presidente por mandato
mand_pr_vl = pd.merge(gastos_pr_grp, mandato_pr, on = 'presidente')
mand_pr_vl['mand_pr_vl'] = mand_pr_vl['valor_atualizado'] / mand_pr_vl['mandatos']
mand_pr_vl.sort_values(by=['mand_pr_vl'], ascending = False).reset_index(drop=True)
mand_pr_vl

,presidente,valor_atualizado,ano,mandatos,mand_pr_vl
0,Lula,108.82,8,2.0,54.410000
1,Dilma,57.47,6,1.5,38.313333
2,Bolsonaro,31.54,4,1.0,31.540000
3,Temer,11.62,2,0.5,23.240000


In [78]:
# tipo de despesa por presidente
despesa_pr = df_merged.groupby(['presidente', 'subelemento_despesa'])['valor_atualizado'].sum().div(1000000).round(2).sort_values(ascending=False).reset_index()
despesa_pr_piv = pd.pivot_table(despesa_pr, values = 'valor_atualizado', index = 'subelemento_despesa', columns = 'presidente', aggfunc = 'sum', fill_value = 0).reset_index()
despesa_pr_piv['total'] = despesa_pr_piv.sum(axis=1)

C:\Users\felip\AppData\Local\Temp\ipykernel_13992\2442281578.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  despesa_pr_piv['total'] = despesa_pr_piv.sum(axis=1)


In [83]:
# calculando percentuais por presidente
for presidente in despesa_pr_piv.columns[1:5]:
    despesa_pr_piv[presidente+'%'] = (despesa_pr_piv[presidente] / despesa_pr_piv['total'] * 100).round(2)
    
col_list = ['Bolsonaro%', 'Dilma%', 'Lula%', 'Temer%']
despesa_pr_piv['total%'] = despesa_pr_piv[col_list].sum(axis=1)
despesa_pr_piv.sort_values(by='total', ascending=False).head(10)


presidente,subelemento_despesa,Bolsonaro,Dilma,Lula,Temer,total,Bolsonaro%,Dilma%,Lula%,Temer%,total%
32,HOSPEDAGENS,15.69,30.21,48.85,4.28,99.03,15.84,30.51,49.33,4.32,100.00
35,LOCACAO DE MEIOS DE TRANSPORTE,0.04,0.19,30.49,0.00,30.72,0.13,0.62,99.25,0.00,100.00
31,GENEROS DE ALIMENTA€ÇO,5.43,9.46,10.32,3.04,28.25,19.22,33.49,36.53,10.76,100.00
28,FORNECIMENTO DE ALIMENTA€ÇO,6.26,6.96,8.69,1.69,23.60,26.53,29.49,36.82,7.16,100.00
87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",1.78,6.47,2.44,2.12,12.81,13.90,50.51,19.05,16.55,100.01
98,SERVI€OS DE TELECOMUNICA€åES,0.02,0.59,1.77,0.01,2.39,0.84,24.69,74.06,0.42,100.01
11,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,0.75,0.37,0.77,0.08,1.97,38.07,18.78,39.09,4.06,100.00
59,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,0.23,0.70,0.88,0.08,1.89,12.17,37.04,46.56,4.23,100.00
36,LOCA€ÇO BENS MOV. OUTR. NATUREZA E INTANGÖVEIS,0.77,0.25,0.58,0.00,1.60,48.12,15.62,36.25,0.00,99.99
56,MATERIAL DE COPA E COZINHA,0.06,0.23,0.38,0.05,0.72,8.33,31.94,52.78,6.94,99.99


In [82]:
# tipo de despesa mais frequente
despesa_prF = df_merged.groupby(['presidente', 'subelemento_despesa'])['valor_atualizado'].count().sort_values(ascending=False).reset_index()
despesa_prF_piv = pd.pivot_table(despesa_prF, values = 'valor_atualizado', index = 'subelemento_despesa', columns = 'presidente', aggfunc = 'sum', fill_value = 0).reset_index()
despesa_prF_piv['total'] = despesa_prF_piv.sum(axis=1)
despesa_prF_piv.sort_values(by='total', ascending=False).head(10)

C:\Users\felip\AppData\Local\Temp\ipykernel_13992\1618395957.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  despesa_prF_piv['total'] = despesa_prF_piv.sum(axis=1)


presidente,subelemento_despesa,Bolsonaro,Dilma,Lula,Temer,total
32,HOSPEDAGENS,1281,5942,37648,419,45290
31,GENEROS DE ALIMENTA€ÇO,7449,7898,4460,5116,24923
28,FORNECIMENTO DE ALIMENTA€ÇO,958,1586,3660,394,6598
98,SERVI€OS DE TELECOMUNICA€åES,34,1818,4007,61,5920
87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",835,1742,1631,674,4882
84,PEDAGIOS,120,1248,1493,172,3033
11,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,652,635,1380,328,2995
59,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,745,820,1131,118,2814
35,LOCACAO DE MEIOS DE TRANSPORTE,21,3,1986,0,2010
66,MATERIAL FARMACOLOGICO,221,1173,264,10,1668


In [84]:
# calculando percentuais por presidente
for presidente in despesa_prF_piv.columns[1:5]:
    despesa_prF_piv[presidente+'%'] = (despesa_prF_piv[presidente] / despesa_prF_piv['total'] * 100).round(2)
    
col_list = ['Bolsonaro%', 'Dilma%', 'Lula%', 'Temer%']
despesa_prF_piv['total%'] = despesa_prF_piv[col_list].sum(axis=1)
despesa_prF_piv.sort_values(by='total', ascending=False).head(10)


presidente,subelemento_despesa,Bolsonaro,Dilma,Lula,Temer,total,Bolsonaro%,Dilma%,Lula%,Temer%,total%
32,HOSPEDAGENS,1281,5942,37648,419,45290,2.83,13.12,83.13,0.93,100.01
31,GENEROS DE ALIMENTA€ÇO,7449,7898,4460,5116,24923,29.89,31.69,17.90,20.53,100.01
28,FORNECIMENTO DE ALIMENTA€ÇO,958,1586,3660,394,6598,14.52,24.04,55.47,5.97,100.00
98,SERVI€OS DE TELECOMUNICA€åES,34,1818,4007,61,5920,0.57,30.71,67.69,1.03,100.00
87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",835,1742,1631,674,4882,17.10,35.68,33.41,13.81,100.00
84,PEDAGIOS,120,1248,1493,172,3033,3.96,41.15,49.23,5.67,100.01
11,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,652,635,1380,328,2995,21.77,21.20,46.08,10.95,100.00
59,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,745,820,1131,118,2814,26.47,29.14,40.19,4.19,99.99
35,LOCACAO DE MEIOS DE TRANSPORTE,21,3,1986,0,2010,1.04,0.15,98.81,0.00,100.00
66,MATERIAL FARMACOLOGICO,221,1173,264,10,1668,13.25,70.32,15.83,0.60,100.00
